In [ ]:
import pandas as pd
from tqdm import tqdm

# Rate the paper

In [ ]:
gs_df = pd.read_csv('./data/gs_results.csv')
gs_df['title'] = gs_df['title'].str.casefold()
pc_df = pd.read_csv('./data/pc_results.csv')
pc_df['title'] = pc_df['title'].str.casefold()
ss_df = pd.read_csv('./data/ss_results.csv')
ss_df['title'] = ss_df['title'].str.casefold()

In [ ]:
gs_df.head()

In [ ]:
pc_df.head()

In [ ]:
ss_df.head()

In [ ]:
TIER1 = 40
TIER2 = 100
TIER3 = 200
gs_rate_df = gs_df[['area','rank','title']][gs_df['rank']<=200].copy()
pc_rate_df = pc_df.copy()
ss_rate_df = ss_df[['area','rank','title']].copy()
gs_rate_df['rate'] = gs_rate_df['rank'].apply(lambda x: 5 if x<=TIER1 else 4 if x<=TIER2 else 3 if x<=TIER3 else 2)
pc_rate_df['rate'] = pc_rate_df['rank'].apply(lambda x: 5 if x<=TIER1 else 4 if x<=TIER2 else 3)
ss_rate_df['rate'] = ss_rate_df['rank'].apply(lambda x: 5 if x<=TIER1 else 4 if x<=TIER2 else 3)

rate_df = gs_rate_df.merge(pc_rate_df, on=['area', 'title'], how='outer', suffixes=('_gs', '_pc'))
rate_df = rate_df.merge(ss_rate_df, on=['area', 'title'], how='outer')
rate_df.rename(columns={'rank': 'rank_ss', 'rate': 'rate_ss'}, inplace=True)
rate_df['rate_ss'].fillna(1, inplace=True)
rate_df['rank_ss'].fillna(201, inplace=True)
rate_df['rate_pc'].fillna(1, inplace=True)
rate_df['rank_pc'].fillna(201, inplace=True)
rate_df['rate_gs'].fillna(1, inplace=True)
rate_df['rank_gs'].fillna(401, inplace=True)

rate_df['agg_rate'] = (rate_df['rate_gs']*0.3 + rate_df['rate_pc']*0.4 + rate_df['rate_ss']*0.3)
rate_df['agg_rank'] = (rate_df['rank_gs']*0.3 + rate_df['rank_pc']*0.4 + rate_df['rank_ss']*0.3)
rate_df = rate_df.sort_values(by=['area', 'agg_rate', 'agg_rank'], ascending=[True, False, True]).reset_index(drop=True)
rate_df['rank'] = rate_df.groupby('area').cumcount()+1

In [ ]:
rate_df.head()

# Get the total dataframe that contains download information and the pdf link

In [ ]:
import requests

In [196]:
total_df = rate_df.merge(gs_df[['area', 'title', 'pdf_link']], on=['area', 'title'], how='left')
total_df = total_df.merge(ss_df[['area', 'title', 'abstract', 'citationCount', 'referenceCount', 'influentialCitationCount', 'fieldsOfStudy']], on=['area', 'title'], how='left')

In [197]:
total_df['pdf_link'].fillna('', inplace=True)
total_df['abstract'].fillna('', inplace=True)

In [223]:
for idx, paper in tqdm(total_df.iterrows(), total=total_df.shape[0]):
    if paper['abstract'] == '':
        title_formatted = paper['title'].replace(' ', '+')
        data = requests.get(f'https://api.semanticscholar.org/graph/v1/paper/search?query={title_formatted}&limit=1&fields=abstract,referenceCount,citationCount,influentialCitationCount').json()
        if data.get('message') is not None:
            print(data.get('message'))
            break
        else:
            data = data.get('data')
            if data:
                data = data[0]
                paper['abstract'] = data.get('abstract')
                paper['citationCount'] = data.get('citationCount')
                paper['referenceCount'] = data.get('referenceCount')
                paper['influentialCitationCount'] = data.get('influentialCitationCount')
    total_df.loc[idx] = paper
        

 28%|██▊       | 3564/12623 [03:15<08:16, 18.25it/s]  


ProxyError: HTTPSConnectionPool(host='api.semanticscholar.org', port=443): Max retries exceeded with url: /graph/v1/paper/search?query=an+improvement+to+the+nearest+neighbor+classifier+and+face+recognition+experiments&limit=1&fields=abstract,referenceCount,citationCount,influentialCitationCount (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 503 Service Unavailable')))

In [224]:
sum(total_df['abstract'] == '')

4750

In [225]:
test_df = total_df[total_df['rank'] <= 100].copy()

In [231]:
for idx, paper in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    if paper['abstract'] == '':
        title_formatted = paper['title'].replace(' ', '+')
        data = requests.get(f'https://api.semanticscholar.org/graph/v1/paper/search?query={title_formatted}&limit=1&fields=abstract,referenceCount,citationCount,influentialCitationCount').json()
        if data.get('message') is not None:
            print(data.get('message'))
            break
        else:
            data = data.get('data')
            if data:
                data = data[0]
                paper['abstract'] = data.get('abstract')
                paper['citationCount'] = data.get('citationCount')
                paper['referenceCount'] = data.get('referenceCount')
                paper['influentialCitationCount'] = data.get('influentialCitationCount')
    test_df.loc[idx] = paper
        

100%|██████████| 3024/3024 [02:33<00:00, 19.72it/s]


In [236]:
test_df['abstract'].fillna('', inplace=True)
sum(test_df['abstract'] == '')

1

In [245]:
test_df.loc[8080, 'title'] = 'An Aerial Weed Detection System for Green Onion Crops Using the You-Only-Look- Once (YOLO) Deep Learning Algorithm'.lower()

In [246]:
test_df.loc[8080, 'abstract'] = 'Herbicide application is a common and inevitable method for preventing weed growth for some crops. Green onions are vulnerable to and significantly affected by weed infestation. However, herbicide contamination can pose as a food safety concern, especially in Japanese cuisine where green onions are typically eaten fresh. As a possible solution, an herbicide spraying system precisely targeting weeds while avoiding green onions was conceptualized. As a preliminary investigation, this study develops and evaluates the performance of what is referred to as the YOLO-WEED, a system that allows the smart detection of weeds through the utilization of unmanned aerial vehicles (UAVs) combined with You-Only-Look-Once (YOLO) deep learning algorithm. YOLO is a forerunner in terms of inference time in object detection, making it suitable for UAV applications. For the dataset, a five-minute UAV video was taken at altitude 4-5 meters at 0-1.3 m/s speed. Each frame from the UAV video were captured and cropped into tiles. 600 of these tiles were selected, annotated and split into training and validation dataset (450) and testing (150). After that, training, validation and testing were performed on YOLO-WEED with the GPU NVIDIA GeForce GTX 1060. IoU, which is the ratio between area of overlap and area of union of the bounding boxes of the ground truth object and the prediction, is the basis of true positive (TP), false positive (FP) and false negative (FN). Based on the TP, FP and FN, the following main performance metrics can be calculated: F1 score (with values 0 to 1) and mean average precision (with values 0 to 100 % with a threshold of 50% for IoU). Moreover, the detection speed expressed in frame per second (FPS) was also determined. YOLO-WEED demonstrated high detection speed (23.7 to 27.8 FPS) and remarkable performance, with mean average precision of 91.09 % and an F1 score of 0.85. YOLO-WEED was also tested on a cropped UAV video and the limitation of YOLO in detecting small objects was minimized. These results successfully show the effectiveness of the YOLO-WEED system for real-time UAV weed detection given its high speed and high accuracy in detection.'

In [ ]:
test_df.to_csv('test_df.csv', index=False)